# Exploring cosmological simulations with CosmoSim
## Part 1: Introduction to job handling with uws-client

CosmoSim is a web application available at http://www.cosmosim.org/ where data from cosmological simulations is available. This includes catalogues of dark matter halos (clusters) and galaxies for different time steps during the evolution of the simulated universe, merging information, substructure data, density fields and more.

In this tutorial, we will use the [uws-client](https://github.com/aipescience/uws-client) for connecting with CosmoSim's UWS-interface for seeing your list of jobs, sending jobs and retrieving results.

## Imports
Import the UWS module from the uws-client:

In [16]:
# import sys
# sys.path.append('<your own path>/uws-client')

from uws import UWS

## Setup the connection

The URL for connecting with CosmoSim's uws-client is 'https://www.cosmosim.org/uws/query/'. You also need to define your username and password, either by inserting it directly below or by saving your credentials in a local cosmosim-user.json file and reading it here. The credentials are the same as on the CosmoSim webpage. If you do not have an account yet, please register at [CosmoSim registration](https://www.cosmosim.org/auth/registration/register). Alternatively, you can use the user `uwstest` with password `gavo` for testing purposes. (Be aware that anyone can use this user and delete your results at any time!)


In [17]:
# set credentials here:
# username = 'uwstest'
# password = 'gavo'

# or read them from a file:
import json
with open('cosmosim-user.json') as credentials_file:    
    username, password = json.load(credentials_file).values()

url = 'https://www.cosmosim.org/uws/query/'
c = UWS.client.Client(url, username, password)

## List previous jobs
Once the connection is set up, you can retrieve the list of previously run jobs with `c.get_job_list()`. You can also provide extra filters for the job list, e.g. filtering by phase, creation time of the job or just output the most recent ones using the `last` keyword.


In [18]:
filters = {'phases': ['PENDING', 'COMPLETED'], 'last': 5}
jobs = c.get_job_list(filters)
print jobs

Job '1489146266078541581' in phase 'PENDING' created at '2017-03-10T12:44:26+01:00' - https://www.cosmosim.org/uws/query/1489146266078541581
Job '1489145816018354009' in phase 'PENDING' created at '2017-03-10T12:36:56+01:00' - https://www.cosmosim.org/uws/query/1489145816018354009
Job '1489145811420913586' in phase 'PENDING' created at '2017-03-10T12:36:51+01:00' - https://www.cosmosim.org/uws/query/1489145811420913586
Job '1489145803552326504' in phase 'PENDING' created at '2017-03-10T12:36:43+01:00' - https://www.cosmosim.org/uws/query/1489145803552326504
Job '1489145792448936116' in phase 'PENDING' created at '2017-03-10T12:36:32+01:00' - https://www.cosmosim.org/uws/query/1489145792448936116



For each job, its unique id, ownerId, creationTime and phase are stored within the job list:

In [19]:
for job in jobs.job_reference:
    print job.id, job.ownerId, job.creationTime, job.phase[0]

1489146266078541581 uwstest 2017-03-10T12:44:26+01:00 PENDING
1489145816018354009 uwstest 2017-03-10T12:36:56+01:00 PENDING
1489145811420913586 uwstest 2017-03-10T12:36:51+01:00 PENDING
1489145803552326504 uwstest 2017-03-10T12:36:43+01:00 PENDING
1489145792448936116 uwstest 2017-03-10T12:36:32+01:00 PENDING


## Create and run a job

For creating a new job, first define the necessary parameters. For CosmoSim this is 'query', which is the SQL string and the optional parameters 'queue' (long or short) and 'table' (a unique table name). We set here an SQL query that will select the 10 most massive clusters from the MDPL2 simulation, Rockstar catalog.

In [20]:
parameters = {'query': 
              'SELECT rockstarId, x,y,z, Mvir FROM MDPL2.Rockstar'\
                + 'WHERE snapnum=125 ORDER BY Mvir DESC LIMIT 10',
              'queue': 'short'}

Now create a new job with these parameters:

In [22]:
job = c.new_job(parameters)

The job is created now. It is not started yet - you can still adjust its parameters with job.parameters and you can check its properties.

Parameter id 'queue' byRef: False is_post: False - value: short
